In [1]:
import os

In [2]:
# define name of SRA file
SRAname='SRR7946391'

#Get file from SRA accession
!prefetch $SRAname


2019-10-11T15:57:10 prefetch.2.8.2: 1) Downloading 'SRR7946391'...
2019-10-11T15:57:10 prefetch.2.8.2:  Downloading via https...
2019-10-11T15:57:12 prefetch.2.8.2: 1) 'SRR7946391' was downloaded successfully
2019-10-11T15:57:12 prefetch.2.8.2: 'SRR7946391' has 0 unresolved dependencies


In [3]:
ls -sh

total 40K
4.0K apt.txt          8.0K ncbi/      8.0K viraq_kaiju-Copy1.ipynb
4.0K environment.yml  4.0K README.md   12K viraq_kaiju.ipynb


In [4]:
#setting up directories
!mkdir -p QC
#input file directory
SRAfiles='ncbi/public/sra'
#quality check path
QC1_sub='QC/'+SRAname+'_sub'

In [5]:
#convert to fastq
os.chdir(SRAfiles)
!fastq-dump $SRAname

Read 85711 spots for SRR7946391
Written 85711 spots for SRR7946391


In [6]:
os.chdir('/home/jovyan')
fastqfile=SRAfiles+'/'+SRAname +'.fastq'
#subsampling
subsample=SRAfiles+'/'+SRAname+'_sub.fq.gz'
!seqtk sample $fastqfile 5000 | gzip > $subsample
!nanoQC -o $QC1_sub $subsample

In [7]:
from IPython.display import IFrame
IFrame(src=QC1_sub +'/nanoQC.html', width=900, height=900)

In [ ]:
# do taxonomic assessment

In [1]:
!mkdir -p kaiju/db
cd kaiju/db
!kaiju-makedb -s viruses

In [7]:
os.chdir ('/home/jovyan')

merged.dmp  names.dmp  nodes.dmp  taxdump.tar.gz  viruses/


In [124]:
#can test with smaller virus database to run faster
!mkdir kaiju_results
kaijudb2='kaiju_db_viruses.fmi'
kaijudbpath2='kaiju/db/viruses'+'/'+kaijudb2

kaijuresults_sub2= 'kaiju_results/'+SRAname+'_sub_viralref'
kronaresults_sub2= 'kaiju_results/'+SRAname+'_Krona_sub_viralref'
kronasub2html='kaiju_results/'+SRAname+'_Krona_sub_viralref.html'

!kaiju -t kaiju/db/nodes.dmp -f $kaijudbpath2 -i $subsample -z 8 -a greedy -e 5 -v -o $kaijuresults_sub2
!kaiju2krona -t kaiju/kaijudb/nodes.dmp -n kaiju/kaijudb/names.dmp -i $kaijuresults_sub2 -o $kronaresults_sub2
!ktImportText -o $kronasub2html $kronaresults_sub2

In [135]:
IFrame(src=kronasub2html, width=900, height=900)

In [ ]:
kaiju2krona -t nodes.dmp -n names.dmp -i /home/kai/Assembly_tests/kaiju_results/SRR7946391 -o /home/kai/Assembly_tests/kaiju_results/SRR7946391_krona
#switch to Krona environment:
ktImportText -o /home/kai/Assembly_tests/kaiju_results/SRR7946391_krona.html /home/kai/Assembly_tests/kaiju_results/SRR7946391_krona
#Add Taxon names to result table:
kaiju-addTaxonNames -t nodes.dmp -n names.dmp -i /home/kai/Assembly_tests/kaiju_results/SRR7946391 -o /home/kai/Assembly_tests/kaiju_results/SRR7946391_names 
